# 函数调用
大模型的函数调用（Function calling），其实就是工具调用，在整个agent的结构中，工具的调用是很重要的一环，在感知环境和执行大模型给出的决策时都充当了重要的角色，下面我们利用天气查询的函数调用来举例说明。

在函数调用中，我们使用大模型目的是为了让它判断出用户当前的问题是否触及了“查询天气”、“查询时间”等情况，最终输出的是调用的函数名字，通过对应函数中搜索天气的举动，得到准确的天气数据，由于这些数据多而繁杂，可以将其交给大模型加工处理，最终就能输出成我们需要的对话。

那么首先我们来设计工具函数，这里仅举例，因此没有调用天气API，如果想深入了解如何操作，可以查看[阿里云平台](https://help.aliyun.com/zh/model-studio/qwen-function-calling)并且查看如何利用API调用天气数据。

In [5]:
### 模型加载
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.01it/s]


## 1. 定义工具函数

In [1]:
import random
from datetime import datetime, timedelta

# 模拟查询天气，不用API调用，直接输出{地点}今天是{天气}
def get_current_weather(arguments):
    # 定义备选天气选项（固定好的）
    weather_conditions=["晴天", "阴天", "小雨", "大雨", "雪天", "多云"]
    # 随机选择一个天气
    weather = random.choice(weather_conditions)
    # 获取地点信息
    location = arguments["location"]
    # 输出内容
    return f"{location}今天是{weather}"

# 模拟查询当前时间工具，不使用API调用，直接输出现在是{时间}
def get_current_time(arguments):
    # 获取当前时间
    now = datetime.now()
    # 格式化时间为字符串
    current_time = now.strftime("%Y年%m月%d日 %H:%M:%S")
    # 输出内容
    return f"现在是{current_time}"

In [2]:
## 测试上面的工具输出
example_arguments = {"location":"北京"}
print(get_current_weather(example_arguments))
print(get_current_time({}))

北京今天是晴天
现在是2025年07月30日 17:46:40


## 2. 创建tools数组
人类在选择工具之前，需要对工具有全面的了解，包括工具的功能、何时使用以及输入参数等。大模型也需要这些信息才能更准确地选择工具。通常来说，对于现在的大模型都会有对应的对话模板，其中会添加Function Calling的模板转换，那么我们只需要按照json模板要求的格式对输入参数处理即可，具体的json模板可以在[Qwen官网](https://qwen.readthedocs.io/zh-cn/latest/framework/function_call.html)找到，其他的模型可能会有不同，需要查找对应的官网文档教程来设计，这里我们仅看Qwen的json格式模板：

In [3]:
tools=[{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "当你想查询指定城市的天气时非常有用。",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                }
            },
            "required": ["location"]
        }
    }
},
{
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
        }
    },
]

In [4]:
## 测试工具
tool_name = [tool["function"]["name"] for tool in tools]
print(f"创建了{len(tools)}个工具，为：{tool_name}\n")

创建了2个工具，为：['get_current_weather', 'get_current_time']



## 3. 融入对话模板
尽管在创建 tools 数组时已经对工具的作用与何时使用工具进行了描述，但在 System Message 中强调何时调用工具通常会提高工具调用的准确率。也就是说我们需要对message中的system进行修改：

In [6]:
prompt="你好，现在几点了，北京天气如何"

In [8]:
### 推理代码
messages = [
    {
        "role": "system",
        "content": """你是一个很有帮助的助手。如果用户提问关于天气的问题，请调用 ‘get_current_weather’ 函数;
     如果用户提问关于时间的问题，请调用‘get_current_time’函数。
     请以友好的语气回答问题。""",
    },
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    tools=tools,
)
print(f"输入文本：{text}\n")


输入文本：<|im_start|>system
你是一个很有帮助的助手。如果用户提问关于天气的问题，请调用 ‘get_current_weather’ 函数;
     如果用户提问关于时间的问题，请调用‘get_current_time’函数。
     请以友好的语气回答问题。

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "get_current_weather", "description": "当你想查询指定城市的天气时非常有用。", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "城市或县区，比如北京市、杭州市、余杭区等。"}}, "required": ["location"]}}}
{"type": "function", "function": {"name": "get_current_time", "description": "当你想知道现在的时间时非常有用。"}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
你好，现在几点了，北京天气如何<|im_end|>
<|im_start|>assistant




## 4. 运行

可以看到function calling部分在模板中体现，而一旦我们问到天气、时间相关的问题的时候，就能触发相应的函数，最终模型输出的是调用的是具体哪一个函数名称，当然如果我们询问触及多个函数，也会返回多个函数，那么话不多说，我们下面看下实际效果：

In [9]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=128,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

<tool_call>
{"name": "get_current_time", "arguments": {}}
</tool_call>
<tool_call>
{"name": "get_current_weather", "arguments": {"location": "北京"}}
</tool_call>


In [10]:
## 提取函数调用的内容
import re
import json

# 1. 用正则一次性抓取 <tool_call>...</tool_call> 里的内容
# 1) 正则匹配所有 JSON 字符串
json_strs = re.findall(r'<tool_call>\s*(\{.*?\})\s*</tool_call>', response, flags=re.S)

# 2) 转成 Python 字典列表
calls = [json.loads(js) for js in json_strs]

print(calls)

[{'name': 'get_current_time', 'arguments': {}}, {'name': 'get_current_weather', 'arguments': {'location': '北京'}}]


In [11]:
# ---------- 2. 函数映射 ----------
function_mapper = {
    "get_current_weather": get_current_weather,
    "get_current_time": get_current_time
}

# ---------- 3. 批量执行并收集结果 ----------
results = []
for call in calls:
    func_name = call["name"]
    args = call["arguments"]
    func = function_mapper[func_name]
    output = func(args)
    results.append({
        "name": func_name,
        "arguments": args,
        "output": output
    })
    print(f"工具[{func_name}] 输出：{output}")

results

工具[get_current_time] 输出：现在是2025年07月30日 17:55:04
工具[get_current_weather] 输出：北京今天是晴天


[{'name': 'get_current_time',
  'arguments': {},
  'output': '现在是2025年07月30日 17:55:04'},
 {'name': 'get_current_weather',
  'arguments': {'location': '北京'},
  'output': '北京今天是晴天'}]

In [12]:
### 对回答加工处理
new_prompt="""
用户提问的内容是{question}
模型给出的回答是{response}
请你根据用户提问的内容和模型给出的回答，组合成合适的回答用来回答用户的问题。
"""
new_prompt= new_prompt.format(
    question=prompt,
    response=results
)
print(f"新的提示：{new_prompt}")

新的提示：
用户提问的内容是你好，现在几点了，北京天气如何
模型给出的回答是[{'name': 'get_current_time', 'arguments': {}, 'output': '现在是2025年07月30日 17:55:04'}, {'name': 'get_current_weather', 'arguments': {'location': '北京'}, 'output': '北京今天是晴天'}]
请你根据用户提问的内容和模型给出的回答，组合成合适的回答用来回答用户的问题。



In [13]:
messages = [
    {"role": "user", "content": new_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

您好，现在是2025年7月30日 17:55:04。北京今天的天气是晴天。


那么很好！你已经成功完成了函数调用的整个流程，这其实是agent中工具起到的作用，贯穿了agent的整个流程。
下面我们提供了几个案例，有兴趣的小伙伴可以尝试实现，需要注意的是对于实时更新的一些资料，使用函数调用会有很好的效果，因为可以有效的避免大模型幻觉。
那么什么时候需要函数调用，什么时候没必要用函数调用，我们用一句话来理解，“能用规则 100% 搞定的，就别劳驾大模型；凡是输入语义多变、输出结构复杂、需要动态组合外部能力的场景，就上函数调用。”